# The General Claims Data
- Primary question of interest: Who was at fault?
<br> <br>
- Secondary questions of interest:
    - What Claim Group should the claim be routed to?
    - Can we predict severity type (minor, moderate, major, life threatening, death)
    - What was the cause of the loss?

In [1]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg

In [2]:
import nltk
from nltk import word_tokenize

In [3]:
from nltk.corpus import stopwords
# stop = stopwords.words('english')

In [104]:
stop = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 
 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 
 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 
 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 
 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 
 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 
 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'doing', 'it', 'how', 'further', 'was', 
 'here', 'than']

In [4]:
import gensim

In [149]:
general_tokens_df = pd.read_csv('data/general_with_tokens.csv')

In [150]:
general_tokens_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,...,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess,claim_level_list,exposure_level_list,accident_desc_list,injury_desc_list,damage_desc_list
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,...,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,"['reviewed', 'coverage', 'sc', 'rated', 'polic...","['none', 'none', 'liability', 'accepted', 'by'...","['insured', 'test', 'driving', 'vehicle', 'ov'...",['neckback'],"['front', 'left', 'side', 'damages', 'towed', ..."
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,...,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,"['c', 'dol', 'tol', 'pm', 'venue', 'fl', 'loca...","['c', 'coverage', 'dol', 'fnol', 'by', 'ni', '...","['iv', 'driving', 'road', 'ov', 'turning', 'la...",['pain'],"['front', 'right', 'headlight', 'front', 'side..."
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,...,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,"['insured', 'hospital', 'lost', 'service', 'mi...","['none', 'none', 'c', 'coverage', 'dol', 'fnol...","['insured', 'stopped', 'stop', 'light', 'cv', ...",['strain'],"['unknown', 'damages', 'rear', 'bumper', 'trun..."
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,...,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,"['insured', 'hospital', 'lost', 'service', 'mi...","['none', 'none', 'hhcsi', 'dos', 'atd', 'brant...","['insured', 'stopped', 'stop', 'light', 'cv', ...",['back'],"['unknown', 'damages', 'rear', 'bumper', 'trun..."
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,...,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,"['adjuster', 'office', 'reassigned', 'adjuster...","['new', 'pip', 'loss', 'received', 'additional...","['ov', 're', 'host', 'veh']","['neck', 'back']","['none', 'none', 'none']"


In [5]:
general_df = pd.read_csv('data/NSS_DS_data.thegeneral.csv')


In [6]:
general_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida


In [7]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120657 entries, 0 to 120656
Data columns (total 14 columns):
ClaimID_RGEN           120657 non-null int64
ExposureID_RGEN        120657 non-null int64
ClaimLevelBody         120657 non-null object
ExposureLevelBody      111199 non-null object
AccidentDescription    120657 non-null object
InjuryDescription      120619 non-null object
DamageDescription      120657 non-null object
SeverityTypeName       87699 non-null object
ClaimGroup             120642 non-null object
FaultRatingName        120656 non-null object
LossCauseName          120657 non-null object
CCCreateTime           120657 non-null object
CloseDate              99611 non-null object
StateName              120657 non-null object
dtypes: int64(2), object(12)
memory usage: 12.9+ MB


In [74]:
general_no_unknown = general_df.loc[general_df.loc[:,'FaultRatingName'] != 'Fault unknown',:].reset_index(drop=True)

In [101]:
general_no_unknown = general_no_unknown.fillna({'ExposureLevelBody':' ', 'InjuryDescription':' '})

In [102]:
general_no_unknown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119272 entries, 0 to 119271
Data columns (total 19 columns):
ClaimID_RGEN                      119272 non-null int64
ExposureID_RGEN                   119272 non-null int64
ClaimLevelBody                    119272 non-null object
ExposureLevelBody                 119272 non-null object
AccidentDescription               119272 non-null object
InjuryDescription                 119272 non-null object
DamageDescription                 119272 non-null object
SeverityTypeName                  86721 non-null object
ClaimGroup                        119257 non-null object
FaultRatingName                   119271 non-null object
LossCauseName                     119272 non-null object
CCCreateTime                      119272 non-null object
CloseDate                         99601 non-null object
StateName                         119272 non-null object
ClaimLevelBody_preprocess         119272 non-null object
ExposureLevelBody_preprocess      1099

In [8]:
nlp = en_core_web_lg.load()

In [9]:
def column_to_list_text(df, col_name):
    return df[col_name].tolist()

In [105]:
def lower_columns(df, col_list):
    n_df = df
    for col_name in col_list:
        n_col = col_name + '_preprocess'
        n_df[n_col] = n_df[col_name].str.lower()
    return n_df

In [106]:
lowered_df = lower_columns(general_no_unknown, ['ClaimLevelBody', 'ExposureLevelBody', 'AccidentDescription', 
                                        'InjuryDescription', 'DamageDescription'])

In [107]:
lowered_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none


In [108]:
def tokenize_claim_level(row):
#     list_cols = [['ClaimLevelBody_preprocess', 'ExposureLevelBody_preprocess', 'AccidentDescription_preprocess', 
#                                         'InjuryDescription_preprocess', 'DamageDescription_preprocess']]
    text = row['ClaimLevelBody_preprocess']
    text_tokenize = [token for token in word_tokenize(text) if token.isalpha() and token not in stop]
    return text_tokenize
        
def pre_process(df, col_list):
    for col in col_list:
        df[col] = df.apply(lambda x: tokenize(x, col), axis=1)
    return df

In [109]:
#lowered_df['ClaimLevelBody_preprocess'] = lowered_df.apply(tokenize_claim_level, axis = 1)

In [110]:
#out_df.head()

In [111]:
lowered_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none


In [79]:
fault_ratings = column_to_list_text(lowered_df, 'FaultRatingName')

In [112]:
claim_level_body = column_to_list_text(lowered_df, 'ClaimLevelBody_preprocess')

In [113]:
exposure_level_body = column_to_list_text(lowered_df, 'ExposureLevelBody_preprocess')

In [114]:
accident_description = column_to_list_text(lowered_df, 'AccidentDescription_preprocess')

In [115]:
injury_description = column_to_list_text(lowered_df, 'InjuryDescription_preprocess')

In [116]:
damage_description = column_to_list_text(lowered_df, 'DamageDescription_preprocess')

In [130]:
claim_level_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in claim_level_body]
exposure_level_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in exposure_level_body]
accident_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in accident_description]
injury_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in injury_description]
damage_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                         for doc in damage_description]

# [token for token in word_tokenize(damage_description) if token.isalpha()]

In [134]:
# claim_level_tokenized[0]

In [138]:
token_cols = {'claim_level_list': claim_level_tokenized,
         'exposure_level_list': exposure_level_tokenized,
         'accident_desc_list': accident_desc_tokenized,
         'injury_desc_list': injury_desc_tokenized,
             'damage_desc_list': damage_desc_tokenized}
df = pd.DataFrame.from_dict(token_cols)

In [139]:
df.head()

,claim_level_list,exposure_level_list,accident_desc_list,injury_desc_list,damage_desc_list
0,"[reviewed, coverage, sc, rated, policy, loss, ...","[none, none, liability, accepted, by, adverse,...","[insured, test, driving, vehicle, ov, pulled, ...",[neckback],"[front, left, side, damages, towed, collision,..."
1,"[c, dol, tol, pm, venue, fl, location, jackson...","[c, coverage, dol, fnol, by, ni, policy, state...","[iv, driving, road, ov, turning, lane, suddenl...",[pain],"[front, right, headlight, front, side, bumper,..."
2,"[insured, hospital, lost, service, middle, cla...","[none, none, c, coverage, dol, fnol, by, ni, p...","[insured, stopped, stop, light, cv, rear, ende...",[strain],"[unknown, damages, rear, bumper, trunk, right,..."
3,"[insured, hospital, lost, service, middle, cla...","[none, none, hhcsi, dos, atd, brant, williams,...","[insured, stopped, stop, light, cv, rear, ende...",[back],"[unknown, damages, rear, bumper, trunk, right,..."
4,"[adjuster, office, reassigned, adjuster, offic...","[new, pip, loss, received, additional, action,...","[ov, re, host, veh]","[neck, back]","[none, none, none]"


In [140]:
len(df)

119272

In [141]:
df_with_tokens = pd.concat([lowered_df, df], axis = 1)

In [143]:
len(df_with_tokens)

119272

In [148]:
# df_with_tokens.to_csv('data/general_with_tokens.csv', index = False)

In [118]:
lowered_df.FaultRatingName.value_counts()


Insured at fault          74304
Other party at fault      29394
Comparative Negligence     8178
No fault                   7395
Name: FaultRatingName, dtype: int64

In [119]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [120]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [121]:
X_train, X_test, y_train, y_test = train_test_split(accident_description, fault_ratings, 
                                                    test_size=0.25, random_state=42)

In [122]:
model = text_clf.fit(X_train, y_train)

In [123]:
predicted_MNB = text_clf.predict(X_test)

In [124]:
predicted_MNB[:5]

array(['Insured at fault', 'Insured at fault', 'Insured at fault',
       'Insured at fault', 'Insured at fault'], dtype='<U22')

In [125]:
np.mean(predicted_MNB == y_test)

0.6397142665504058

In [126]:
len(predicted_MNB)

29818

In [127]:
text_clf.score(X_test, y_test)

0.6397142665504058

In [128]:
from collections import Counter

In [129]:
Counter(predicted_MNB)

Counter({'Insured at fault': 28402,
         'Other party at fault': 912,
         'No fault': 404,
         'Comparative Negligence': 100})

In [98]:
28402/29818

0.9525119055603998

In [99]:
74304/len(general_no_unknown)

0.622979408411027